In [2]:
pip install lightgbm

  Obtaining dependency information for lightgbm from https://files.pythonhosted.org/packages/11/3f/49913ed111286e23bcc40daab54542d80924264dca8ae371514039ab83ab/lightgbm-4.5.0-py3-none-macosx_12_0_arm64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 237.1 kB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

In [ ]:
## DATA ROBOT ##

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb

In [3]:
# Read training and testing data for S&P 500
train_data_lgb = pd.read_excel("/Users/shannenwibisono/Desktop/-SKRIPSI-/S&P500/SP500 Training Testing.xlsx", sheet_name="Training Data")
test_data_lgb = pd.read_excel("/Users/shannenwibisono/Desktop/-SKRIPSI-/S&P500/SP500 Training Testing.xlsx", sheet_name="Testing Data")

# Prepare features and target for training
x_train_lgb = train_data_lgb[['Price.', 'Maturity', 'Strike', 'RVlag1', 'RVlag7', 'RVlag15', 'RVlag30', 
                                'BVlag1', 'BVlag7', 'BVlag15', 'BVlag30', 'SJVlag7', 'SJVlag15', 'SJVlag30']].values
y_train_lgb = train_data_lgb['Last'].values

# Prepare features and target for testing
x_test_lgb = test_data_lgb[['Price.', 'Maturity', 'Strike', 'RVlag1', 'RVlag7', 'RVlag15', 'RVlag30', 
                             'BVlag1', 'BVlag7', 'BVlag15', 'BVlag30', 'SJVlag7', 'SJVlag15', 'SJVlag30']].values
y_test_lgb = test_data_lgb['Last'].values

In [11]:
# Define the LightGBM parameters
params = {
    'boost_from_average': True,
    'colsample_bytree': 0.5,
    'fair_c': 1,
    'huber_delta': 1,
    'learning_rate': 0.05,
    'max_bin': 255,
    'max_delta_step': 0.7,
    'max_depth': -1,  # None is represented by -1 in LightGBM
    'min_child_samples': 10,
    'min_child_weight': 5,
    'min_split_gain': 0,
    'n_estimators': 2500,
    'num_leaves': 64,
    'objective': 'gamma',
    'reg_alpha': 0,
    'reg_lambda': 0,
    'subsample': 1,
    'subsample_for_bin': 50000,
    'subsample_freq': 1,
    'tweedie_variance_power': 1.5,
    'early_stopping_round': 200  # Set early stopping in parameters
}

# Create LightGBM datasets
train_set = lgb.Dataset(x_train_lgb, label=y_train_lgb)
test_set = lgb.Dataset(x_test_lgb, label=y_test_lgb, reference=train_set)

# Train the model without separate early_stopping_rounds
model = lgb.train(
    params, 
    train_set, 
    valid_sets=[train_set, test_set], 
    valid_names=["train", "test"], 
    num_boost_round=2500
)

# Predict on test data
y_pred_lgb = model.predict(x_test_lgb, num_iteration=model.best_iteration)

# Calculate RMSE for evaluation
rmse = mean_squared_error(y_test_lgb, y_pred_lgb, squared=False)
print(f"RMSE on test set: {rmse}")

# Calculate MAPE
mape = np.mean(np.abs((y_test_lgb - y_pred_lgb) / y_test_lgb)) * 100
print(f"MAPE on test set: {mape:.2f}%")

/Users/shannenwibisono/anaconda3/lib/python3.11/site-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.008065 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3392
[LightGBM] [Info] Number of data points in the train set: 28473, number of used features: 14
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Start training from score 6.276901
Training until validation scores don't improve for 200 rounds
Did not meet early stopping. Best iteration is:
[2434]	train's gamma: 5.66012	test's gamma: 5.69805
RMSE on test set: 42.34626325920829
MAPE on test set: 8.68%


In [12]:
dates = test_data_lgb['Time']  # Replace 'Date' with the correct column name for the dates

# Prepare the results DataFrame
results = pd.DataFrame({
    'Date': dates,                 # Dates from the test dataset
    'Actual': y_test_lgb,          # Actual values
    'Predicted': y_pred_lgb        # Predicted values
})

# Save the results to an Excel file
output_file = "/Users/shannenwibisono/Desktop/-Skripsi-/Results LGBM SP500 DataRobot Call.xlsx"
results.to_excel(output_file, index=False)

print(f"Results saved to Excel file: {output_file}")

Results saved to Excel file: /Users/shannenwibisono/Desktop/-Skripsi-/Results LGBM SP500 DataRobot Call.xlsx


In [ ]:
### GRIDSEARCHCV ###

In [5]:
# Define a small grid to tune
param_grid = {
    'learning_rate': [0.05, 0.07],     # Slightly vary the learning rate
    'num_leaves': [16, 64],              # Try a few values around 16
    'max_depth': [-1, 10, 20],               # Include a capped depth as options
    'min_child_samples': [5, 10, 20]         # Slight variations in child samples
}

# Define LightGBM regressor
lgb_model = lgb.LGBMRegressor(
    boosting_type='gbdt',
    objective='gamma',
    boost_from_average=True,
    colsample_bytree=0.5,
    fair_c=1,
    huber_delta=1,
    max_bin=255,
    max_delta_step=0.7,
    n_estimators=2500,
    reg_alpha=0,
    reg_lambda=0,
    subsample=1,
    subsample_for_bin=50000,
    subsample_freq=1,
    tweedie_variance_power=1.5
)

# Define RMSE as the scoring function
scorer = make_scorer(mean_squared_error, squared=False)

# Set up GridSearchCV
grid_search = GridSearchCV(estimator=lgb_model, param_grid=param_grid, cv=5, scoring=scorer, verbose=1)

# Run the grid search
grid_search.fit(x_train_lgb, y_train_lgb)

# Output the best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best RMSE Score:", grid_search.best_score_)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000751 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3392
[LightGBM] [Info] Number of data points in the train set: 22778, number of used features: 14
[LightGBM] [Info] Start training from score 6.272116
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001028 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3392
[LightGBM] [Info] Number of data points in the train set: 22778, number of used features: 14
[LightGBM] [Info] Start training from score 6.2

[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001203 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3376
[LightGBM] [Info] Number of data points in the train set: 22778, number of used features: 14
[LightGBM] [Info] Start training from score 6.276158
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001132 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3395
[LightGBM] [Info] Number of data points in the train set: 22779, number of used features: 14
[LightGBM] [Info] Start training from score 6.278050
[Lig

[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000692 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3393
[LightGBM] [Info] Number of data points in the train set: 22779, number of used features: 14
[LightGBM] [Info] Start training from score 6.281510
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000570 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3392
[LightGBM] [Info] Number of data points in the train set: 22778, number of used features: 14
[LightGBM] [Info] Start training from score 6.272116
[Lig

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000814 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3392
[LightGBM] [Info] Number of data points in the train set: 22778, number of used features: 14
[LightGBM] [Info] Start training from score 6.272116
[LightGBM] [Warning] Unknown parameter: huber_delta


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001037 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3393
[LightGBM] [Info] Number of data points in the train set: 22779, number of used features: 14
[LightGBM] [Info] Start traini

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000999 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3393
[LightGBM] [Info] Number of data points in the train set: 22779, number of used features: 14
[LightGBM] [Info] Start training from score 6.281510
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000567 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3392
[LightGBM] [Info] Number of data points in the train set: 22778, number of used features: 14
[LightGBM] [Info] Start training from score 6.272116
[LightGBM] [Warning] Unknown parameter: huber_delta
[Lig

[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000588 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3392
[LightGBM] [Info] Number of data points in the train set: 22778, number of used features: 14
[LightGBM] [Info] Start training from score 6.276648
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000693 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3376
[LightGBM] [Info] Number of data points in the train set: 22778, number of used features: 14
[LightGBM] [Info] Start training from score 6.276158
[Lig

In [6]:
# Step 2: Predict on the test set using the best parameters
best_params = grid_search.best_params_

# Initialize the model with the best parameters
best_model = lgb.LGBMRegressor(**best_params,
                                 boosting_type='gbdt',
                                 objective='gamma',
                                 boost_from_average=True,
                                 colsample_bytree=0.5,
                                 fair_c=1,
                                 huber_delta=1,
                                 max_bin=255,
                                 max_delta_step=0.7,
                                 n_estimators=2500,
                                 reg_alpha=0,
                                 reg_lambda=0,
                                 subsample=1,
                                 subsample_for_bin=50000,
                                 subsample_freq=1,
                                 tweedie_variance_power=1.5)

# Fit the model to the training data
best_model.fit(x_train_lgb, y_train_lgb)

# Predict on the test data
y_pred_lgb = best_model.predict(x_test_lgb)

# Calculate RMSE for evaluation
rmse = mean_squared_error(y_test_lgb, y_pred_lgb, squared=False)
print(f"RMSE on test set: {rmse}")

# Optional: Calculate MAPE
mape = np.mean(np.abs((y_test_lgb - y_pred_lgb) / y_test_lgb)) * 100
print(f"MAPE on test set: {mape:.2f}%")

[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000376 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3392
[LightGBM] [Info] Number of data points in the train set: 28473, number of used features: 14
[LightGBM] [Info] Start training from score 6.276901
[LightGBM] [Warning] Unknown parameter: huber_delta
RMSE on test set: 56.72632143109946
MAPE on test set: 10.38%


In [7]:
best_params

{'learning_rate': 0.05,
 'max_depth': 10,
 'min_child_samples': 5,
 'num_leaves': 16}

In [10]:
dates = test_data_lgb['Time']  # Replace 'Date' with the correct column name for the dates

# Prepare the results DataFrame
results = pd.DataFrame({
    'Date': dates,                 # Dates from the test dataset
    'Actual': y_test_lgb,          # Actual values
    'Predicted': y_pred_lgb        # Predicted values
})

# Save the results to an Excel file
output_file = "/Users/shannenwibisono/Desktop/-Skripsi-/Results LGBM SP500 GridSCV Call.xlsx"
results.to_excel(output_file, index=False)

print(f"Results saved to Excel file: {output_file}")

Results saved to Excel file: /Users/shannenwibisono/Desktop/-Skripsi-/Results LGBM SP500 GridSCV Call.xlsx


In [ ]:
### PUT OPTIONS ###

In [14]:
# Read training and testing data for Put options
train_data_put = pd.read_excel("/Users/shannenwibisono/Desktop/-SKRIPSI-/S&P500/SP500 Training Testing PUT.xlsx", sheet_name="Training Data Put")
test_data_put = pd.read_excel("/Users/shannenwibisono/Desktop/-SKRIPSI-/S&P500/SP500 Training Testing PUT.xlsx", sheet_name="Testing Data Put")

# Prepare features and target for training (Put)
x_train_lgb_put = train_data_put[['Price.', 'Maturity', 'Strike', 'RVlag1', 'RVlag7', 'RVlag15', 'RVlag30', 
                                  'BVlag1', 'BVlag7', 'BVlag15', 'BVlag30', 'SJVlag7', 'SJVlag15', 'SJVlag30']].values
y_train_lgb_put = train_data_put['Last'].values

# Prepare features and target for testing (Put)
x_test_lgb_put = test_data_put[['Price.', 'Maturity', 'Strike', 'RVlag1', 'RVlag7', 'RVlag15', 'RVlag30', 
                                'BVlag1', 'BVlag7', 'BVlag15', 'BVlag30', 'SJVlag7', 'SJVlag15', 'SJVlag30']].values
y_test_lgb_put = test_data_put['Last'].values

In [ ]:
# GRIDSEARCHCV #

In [15]:
# Define a small grid to tune
param_grid = {
    'learning_rate': [0.05, 0.07],     # Slightly vary the learning rate
    'num_leaves': [16, 32],              # Try a few values around 16
    'max_depth': [-1, 10, 20],               # Include a capped depth as options
    'min_child_samples': [5, 10, 20]         # Slight variations in child samples
}

# Define LightGBM regressor
lgb_model = lgb.LGBMRegressor(
    boosting_type='gbdt',
    objective='gamma',
    boost_from_average=True,
    colsample_bytree=0.5,
    fair_c=1,
    huber_delta=1,
    max_bin=255,
    max_delta_step=0.7,
    n_estimators=2500,
    reg_alpha=0,
    reg_lambda=0,
    subsample=1,
    subsample_for_bin=50000,
    subsample_freq=1,
    tweedie_variance_power=1.5
)

# Define RMSE as the scoring function
scorer = make_scorer(mean_squared_error, squared=False)

# Set up GridSearchCV
grid_search = GridSearchCV(estimator=lgb_model, param_grid=param_grid, cv=5, scoring=scorer, verbose=1)

# Run the grid search
grid_search.fit(x_train_lgb_put, y_train_lgb_put)

# Output the best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best RMSE Score:", grid_search.best_score_)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001272 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3424
[LightGBM] [Info] Number of data points in the train set: 30715, number of used features: 14
[LightGBM] [Info] Start training from score 4.687741
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000782 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3420
[LightGBM] [Info] Number of data points in the train set: 30715, number of used features: 14
[LightGBM] [Info] Start training from score 4.6

[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000979 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3419
[LightGBM] [Info] Number of data points in the train set: 30715, number of used features: 14
[LightGBM] [Info] Start training from score 4.682575
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000182 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3417
[LightGBM] [Info] Number of data points in the train set: 30715, number of used featur

[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001083 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3417
[LightGBM] [Info] Number of data points in the train set: 30715, number of used features: 14
[LightGBM] [Info] Start training from score 4.685326
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000754 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3414
[LightGBM] [Info] Number of data points in the train set: 30716, number of used features: 14
[LightGBM] [Info] Start training from score 4.685329
[Lig

[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000507 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3424
[LightGBM] [Info] Number of data points in the train set: 30715, number of used features: 14
[LightGBM] [Info] Start training from score 4.687741
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001075 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3420
[LightGBM] [Info] Number of data points in the train set: 30715, number of used featur

[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001280 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3420
[LightGBM] [Info] Number of data points in the train set: 30715, number of used features: 14
[LightGBM] [Info] Start training from score 4.683239
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000869 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3419
[LightGBM] [Info] Number of data points in the train set: 30715, number of used features: 14
[LightGBM] [Info] Start training from score 4.682575
[LightGBM] [Warning] Unknown parameter: huber_delta
[Lig

...

[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001111 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3424
[LightGBM] [Info] Number of data points in the train set: 30715, number of used features: 14
[LightGBM] [Info] Start training from score 4.687741
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001217 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3420
[LightGBM] [Info] Number of data points in the train set: 30715, number of used features: 14
[LightGBM] [Info] Start training from score 4.683239
[Lig

[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001139 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3419
[LightGBM] [Info] Number of data points in the train set: 30715, number of used features: 14
[LightGBM] [Info] Start training from score 4.682575
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001387 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3417
[LightGBM] [Info] Number of data points in the train set: 30715, number of used features: 14
[LightGBM] [Info] Start training from score 4.685326
[Lig

[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001457 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3414
[LightGBM] [Info] Number of data points in the train set: 30716, number of used features: 14
[LightGBM] [Info] Start training from score 4.685329
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001468 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3424
[LightGBM] [Info] Number of data points in the train set: 30715, number of used features: 14
[LightGBM] [Info] Start training from score 4.687741
[Lig

[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001259 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3420
[LightGBM] [Info] Number of data points in the train set: 30715, number of used features: 14
[LightGBM] [Info] Start training from score 4.683239
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001263 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3419
[LightGBM] [Info] Number of data points in the train set: 30715, number of used features: 14
[LightGBM] [Info] Start training from score 4.682575
[Lig

[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001378 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3417
[LightGBM] [Info] Number of data points in the train set: 30715, number of used features: 14
[LightGBM] [Info] Start training from score 4.685326
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001224 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3414
[LightGBM] [Info] Number of data points in the train set: 30716, number of used features: 14
[LightGBM] [Info] Start training from score 4.685329
[Lig

In [16]:
# Use the best parameters to reinitialize the model
best_params = grid_search.best_params_

# Initialize the model with the best parameters
best_model = lgb.LGBMRegressor(**best_params,
                               boosting_type='gbdt',
                               objective='gamma',
                               boost_from_average=True,
                               colsample_bytree=0.5,
                               fair_c=1,
                               huber_delta=1,
                               max_bin=255,
                               max_delta_step=0.7,
                               n_estimators=2500,
                               reg_alpha=0,
                               reg_lambda=0,
                               subsample=1,
                               subsample_for_bin=50000,
                               subsample_freq=1,
                               tweedie_variance_power=1.5)

# Fit the model to the training data
best_model.fit(x_train_lgb_put, y_train_lgb_put)

# Predict on the test data
y_pred_lgb_put = best_model.predict(x_test_lgb_put)

# Calculate RMSE for evaluation
rmse = mean_squared_error(y_test_lgb_put, y_pred_lgb_put, squared=False)
print(f"RMSE on test set: {rmse}")

# Optional: Calculate MAPE
mape = np.mean(np.abs((y_test_lgb_put - y_pred_lgb_put) / y_test_lgb_put)) * 100
print(f"MAPE on test set: {mape:.2f}%")

[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002059 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3425
[LightGBM] [Info] Number of data points in the train set: 38394, number of used features: 14
[LightGBM] [Info] Start training from score 4.684843
[LightGBM] [Warning] Unknown parameter: huber_delta
RMSE on test set: 31.548439797905726
MAPE on test set: 11.97%


In [19]:
dates = test_data_put['Time']  # Replace 'Date' with the correct column name for the dates

# Prepare the results DataFrame
results = pd.DataFrame({
    'Date': dates,                 # Dates from the test dataset
    'Actual': y_test_lgb_put,          # Actual values
    'Predicted': y_pred_lgb_put        # Predicted values
})

# Save the results to an Excel file
output_file = "/Users/shannenwibisono/Desktop/-Skripsi-/Results LGBM SP500 GridSCV Put.xlsx"
results.to_excel(output_file, index=False)

print(f"Results saved to Excel file: {output_file}")

Results saved to Excel file: /Users/shannenwibisono/Desktop/-Skripsi-/Results LGBM SP500 GridSCV Put.xlsx


In [ ]:
### DATA ROBOT ###

In [20]:
# Define a small grid to tune
param_grid = {
    'learning_rate': [0.05],     # Slightly vary the learning rate
    'num_leaves': [16],          # Try a few values around 64
    'max_depth': [-1],           # Include a capped depth as options
    'min_child_samples': [10]    # Slight variations in child samples
}

# Define LightGBM regressor
lgb_model = lgb.LGBMRegressor(
    boosting_type='gbdt',
    objective='gamma',
    boost_from_average=True,
    colsample_bytree=0.5,
    fair_c=1,
    huber_delta=1,
    max_bin=255,
    max_delta_step=0.7,
    n_estimators=2500,
    reg_alpha=0,
    reg_lambda=0,
    subsample=1,
    subsample_for_bin=50000,
    subsample_freq=1,
    tweedie_variance_power=1.5
)

# Define RMSE as the scoring function
scorer = make_scorer(mean_squared_error, squared=False)

# Set up GridSearchCV
grid_search = GridSearchCV(estimator=lgb_model, param_grid=param_grid, cv=5, scoring=scorer, verbose=1)

# Run the grid search
grid_search.fit(x_train_lgb_put, y_train_lgb_put)

# Output the best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best RMSE Score:", grid_search.best_score_)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001460 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3424
[LightGBM] [Info] Number of data points in the train set: 30715, number of used features: 14
[LightGBM] [Info] Start training from score 4.687741
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001679 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3420
[LightGBM] [Info] Number of data points in the train set: 30715, number of used features: 14
[LightGBM] [Info] Start training from score 4.6832

In [22]:
# Use the best parameters to reinitialize the model
best_params = grid_search.best_params_

# Initialize the model with the best parameters
best_model = lgb.LGBMRegressor(**best_params,
                               boosting_type='gbdt',
                               objective='gamma',
                               boost_from_average=True,
                               colsample_bytree=0.5,
                               fair_c=1,
                               huber_delta=1,
                               max_bin=255,
                               max_delta_step=0.7,
                               n_estimators=2500,
                               reg_alpha=0,
                               reg_lambda=0,
                               subsample=1,
                               subsample_for_bin=50000,
                               subsample_freq=1,
                               tweedie_variance_power=1.5)

# Fit the model to the training data
best_model.fit(x_train_lgb_put, y_train_lgb_put)

# Predict on the test data
y_pred_lgb_put = best_model.predict(x_test_lgb_put)

# Calculate RMSE for evaluation
rmse = mean_squared_error(y_test_lgb_put, y_pred_lgb_put, squared=False)
print(f"RMSE on test set: {rmse}")

# Optional: Calculate MAPE
mape = np.mean(np.abs((y_test_lgb_put - y_pred_lgb_put) / y_test_lgb_put)) * 100
print(f"MAPE on test set: {mape:.2f}%")

[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004693 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3425
[LightGBM] [Info] Number of data points in the train set: 38394, number of used features: 14
[LightGBM] [Info] Start training from score 4.684843
[LightGBM] [Warning] Unknown parameter: huber_delta
RMSE on test set: 26.247843998224333
MAPE on test set: 11.92%


In [23]:
dates = test_data_put['Time']  # Replace 'Date' with the correct column name for the dates

# Prepare the results DataFrame
results = pd.DataFrame({
    'Date': dates,                 # Dates from the test dataset
    'Actual': y_test_lgb_put,          # Actual values
    'Predicted': y_pred_lgb_put        # Predicted values
})

# Save the results to an Excel file
output_file = "/Users/shannenwibisono/Desktop/-Skripsi-/Results LGBM SP500 DataRobot Put.xlsx"
results.to_excel(output_file, index=False)

print(f"Results saved to Excel file: {output_file}")

Results saved to Excel file: /Users/shannenwibisono/Desktop/-Skripsi-/Results LGBM SP500 DataRobot Put.xlsx
